# Swin w/ Fastai + RAPIDS SVR + Yolov5 features

- [Feature Engineering notebook](https://www.kaggle.com/nilavanakilan/petfinder-feature-engineering)
- [Pretrained 10 fold Swin models](https://www.kaggle.com/nilavanakilan/swin-fastai)

**V1** - Train SVR w/o Yolo features and metadata<br>
**V2** - Infer SVR w/o Yolo features and metadata<br>
**V4** - Train SVR w/ Yolo features and metadata<br>
**V5** - Infer SVR w Yolo features and metadata<br>

## References
- https://www.kaggle.com/cdeotte/rapids-svr-boost-17-8/notebook
- https://www.kaggle.com/tanlikesmath/petfinder-pawpularity-eda-fastai-starter

## Load libraries

In [ ]:
import sys
#sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
sys.path.append("../input/timm-3monthsold/pytorch-image-models-master 2")

import pandas as pd
import numpy as np
import timm
import torch.nn as nn
from fastai.vision.all import *
import math
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import gc
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

In [ ]:
seed = 999
BATCH_SIZE = 32
N_FOLDS = 10
set_seed(seed, reproducible=True)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms = True

## Metric

In [ ]:
def petfinder_rmse(input,target):
    return 100*torch.sqrt(F.mse_loss(F.sigmoid(input.flatten()), target))

In [ ]:
# if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
#     os.makedirs('/root/.cache/torch/hub/checkpoints/')
# !cp '../input/swin-transformer/swin_large_patch4_window7_224_22kto1k.pth' '/root/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22kto1k.pth'

## Training data
- Add path using ID
- Drop ID
- Shuffle
- norm_score = Pawpularity/100

In [ ]:
df = pd.read_csv('../input/petfinder-feature-engineering/train_10folds.csv')
df.drop(columns='kfold',inplace=True)
df['path'] = df['Id'].map(lambda x:'../input/petfinder-pawpularity-score/train/'+x+'.jpg')
df = df.drop(columns=['Id'])
df = df.sample(frac=1).reset_index(drop=True)
df['norm_score'] = df['Pawpularity']/100
df.head()

## YOLO Feature Engineering on test data

In [ ]:
def get_image_info(file_path, plot=False):
    img = plt.imread(file_path)
    h, w, c = img.shape
    
    if plot:
        fig, ax = plt.subplots(1, 2, figsize=(8,8))
        ax[0].set_title('Detected pets', size=16)
        ax[0].imshow(img)
        
    results = yolov5x6_model(img, augment=True)

    pet_pixels = np.zeros(shape=[h, w], dtype=np.uint8)

    image_info = { 
        'n_pets': 0,
        'n_dogs': 0,
        'n_cats': 0,
        'labels': [],
        'x_min': 0,
        'x_max': w - 1,
        'y_min': 0,
        'y_max': h - 1,
        'avg_w': 0,
        'avg_h': 0,
        'avg_area': 0
    }
    
    pets_found = []
    
    for x1, y1, x2, y2, threshold, label in results.xyxy[0].cpu().detach().numpy():
        label = results.names[int(label)]
        if label in ['cat', 'dog']:
            image_info['n_pets'] += 1
            image_info['labels'].append(label)
            image_info['x_min'] = max(x1, image_info['x_min'])
            image_info['x_max'] = min(x2, image_info['x_max'])
            image_info['y_min'] = max(y1, image_info['y_min'])
            image_info['y_max'] = min(y2, image_info['y_max'])
            image_info['avg_w'] += abs(x2-x1)
            image_info['avg_h'] += abs(y2-y1)
            image_info['avg_area'] += image_info['avg_w']*image_info['avg_h']
            
            if label == 'cat':
                image_info['n_cats'] += 1
            else:
                image_info['n_dogs'] += 1
            
            pet_pixels[int(y1):int(y2), int(x1):int(x2)] = 1
            
            pets_found.append([x1, x2, y1, y2, label])
            
        res = 0.1 if image_info['n_pets'] == 0 else 0
            
        image_info['avg_w'] /= (image_info['n_pets']+res)
        image_info['avg_h'] /= (image_info['n_pets']+res)
        image_info['avg_area'] /= (image_info['n_pets']+res)

    if plot:
        for x1, x2, y1, y2, label in pets_found:
            c = 'red' if label == 'dog' else 'blue'
            rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=2, edgecolor=c, facecolor='none')
            ax[0].add_patch(rect)
            ax[0].text(max(25, (x2+x1)/2), max(25, y1-h*0.02), label, c=c, ha='center', size=14)
                
    image_info['pet_ratio'] = pet_pixels.sum() / (h*w)

    if plot:
        ax[1].set_title('Pixels Containing Pets', size=16)
        ax[1].imshow(pet_pixels)
        plt.show()
        
    return image_info

In [ ]:
!mkdir /root/.config/Ultralytics/
!cp ../input/yolo-arial/Arial.ttf /root/.config/Ultralytics/Arial.ttf

In [ ]:
yolov5x6_model = torch.hub.load('../input/yolov5', 'custom', source='local', force_reload=True, path='../input/ultralyticsyolov5aweights/yolov5x6.pt')

test_df = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
test_df['Pawpularity'] = [1]*len(test_df)
test_df['path'] = test_df['Id'].map(lambda x:'../input/petfinder-pawpularity-score/test/'+x+'.jpg')

IMAGES_INFO = {
    'Id': [],
    'n_pets': [],
    'n_cats': [],
    'n_dogs': [],
    'label': [],
    'x_min': [],
    'x_max': [],
    'y_min': [],
    'y_max': [],
    'avg_w': [],
    'avg_h': [], 
    'avg_area': [],
    'pet_ratio': []
}

for idx, file_path in tqdm(enumerate(test_df['path']),total=len(test_df)):
    image_info = get_image_info(file_path, plot=False)
    IMAGES_INFO['Id'].append(file_path.split('/')[-1].split('.')[0])
    IMAGES_INFO['n_pets'].append(image_info['n_pets'])
    IMAGES_INFO['n_cats'].append(image_info['n_cats'])
    IMAGES_INFO['n_dogs'].append(image_info['n_dogs'])
    IMAGES_INFO['x_min'].append(image_info['x_min'])
    IMAGES_INFO['x_max'].append(image_info['x_max'])
    IMAGES_INFO['y_min'].append(image_info['y_min'])
    IMAGES_INFO['y_max'].append(image_info['y_max'])
    IMAGES_INFO['avg_w'].append(image_info['avg_w'])
    IMAGES_INFO['avg_h'].append(image_info['avg_h'])
    IMAGES_INFO['avg_area'].append(image_info['avg_area'])
    IMAGES_INFO['pet_ratio'].append(image_info['pet_ratio'])
    
    labels = image_info['labels']
    if len(set(labels)) == 1:
        IMAGES_INFO['label'].append(labels[0])
    elif len(set(labels)) > 1:
        IMAGES_INFO['label'].append(labels[0])
    else:
        IMAGES_INFO['label'].append('unknown')

img_info_test = pd.DataFrame(IMAGES_INFO)
test_df = pd.merge(img_info_test, test_df, on='Id')
test_df = test_df.drop(columns=['Id'])

le = LabelEncoder()
test_df['label'] = le.fit_transform(test_df['label'])
test_df.head()

## Create folds

In [ ]:
num_bins = int(np.ceil(2*((len(df))**(1./3))))
df['bins'] = pd.cut(df['norm_score'], bins=num_bins, labels=False)
df['fold'] = -1

strat_kfold = StratifiedKFold(n_splits=N_FOLDS, random_state=seed, shuffle=True)
for i, (_, train_index) in enumerate(strat_kfold.split(df.index, df['bins'])):
    df.iloc[train_index, -1] = i
    
df['fold'] = df['fold'].astype('int')

## Import RAPIDS

In [ ]:
import cuml, pickle
from cuml.svm import SVR
print('RAPIDS version',cuml.__version__,'\n')

# set LOAD_SVR_FROM_PATH = None to train SVR 
LOAD_SVR_FROM_PATH = '../input/petfinder-swin-svr-fastai/'

print('Train shape:', df.shape )
df.head()

# Train/Infer SVR with Swin embeddings

In [ ]:
super_final_predictions = []
super_final_predictions2 = []
super_final_oof_predictions = []
super_final_oof_predictions2 = []
super_final_oof_true = []

for i in range(N_FOLDS):

    print('='*10,f'Fold {i}','='*10)
    
    df_test = test_df.copy()
    df_valid = df[df.fold == i].reset_index(drop=True)
    
    dense_features = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 
                       'Accessory', 'Group', 'Collage', 'Human', 'Occlusion',
                       'Info', 'Blur','n_pets', 'n_dogs', 'n_cats', 'label', 
                       'pet_ratio', 'avg_w', 'avg_h', 'avg_area']
    
    learn = load_learner(fname = Path(f'../input/swin-fastai/model_fold_{i}.pkl'), cpu=False)
    
    print('NN predicting test...')
    test_dl = learn.dls.test_dl(df_test)
    nn_preds, _ = learn.get_preds(dl=test_dl)
    test_preds = np.array([])
    for preds in nn_preds:
        test_preds = np.concatenate([test_preds,preds],axis=0)   
    super_final_predictions.append(test_preds)
    
    print('NN predicting OOF...')
    valid_dl = learn.dls.test_dl(df_valid)
    nn_preds, _ = learn.get_preds(dl=valid_dl)
    oof_preds = np.array([])
    for preds in nn_preds:
        oof_preds = np.concatenate([oof_preds,preds],axis=0)   
    super_final_oof_predictions.append(oof_preds)
    
    learn.model.head = nn.Identity()
    
    name = f"SVR_fold_{i}.pkl" 
    if LOAD_SVR_FROM_PATH is None:
        
        df_train = df[df.fold != i].reset_index(drop=True)
        
        print('Extracting train embedding...')
        train_dl = learn.dls.test_dl(df_train)
        train_preds, _ = learn.get_preds(dl=train_dl)
        
        embed = np.c_[train_preds.numpy(),df_train[dense_features].values]
        
        clf = SVR()
        clf.fit(embed,df_train.norm_score.values.astype('float32'))
        
        pickle.dump(clf, open(name, "wb"))
    else:
        clf = pickle.load(open(LOAD_SVR_FROM_PATH+name, "rb"))
    
    print('NN+SVR predicting test...')
    test_dl = learn.dls.test_dl(df_test)
    test_preds, _ = learn.get_preds(dl=test_dl)
    embed = np.c_[test_preds.numpy(),df_test[dense_features].values]
    super_final_predictions2.append(clf.predict(embed))
    
    print('NN+SVR predicting OOF...')
    valid_dl = learn.dls.test_dl(df_valid)
    oof_preds, _ = learn.get_preds(dl=valid_dl)
    embed = np.c_[oof_preds.numpy(),df_valid[dense_features].values]
    super_final_oof_predictions2.append(clf.predict(embed))
    
    final_oof_true = df_valid.norm_score.values
    super_final_oof_true.append(final_oof_true)
    
    rsme = 100*np.sqrt(np.mean((super_final_oof_true[-1] - np.array(super_final_oof_predictions[-1]))**2.0))
    print('NN RSME =',rsme,'\n')
    rsme = 100*np.sqrt(np.mean((super_final_oof_true[-1] - np.array(super_final_oof_predictions2[-1]))**2.0))
    print('NN+SVR RSME =',rsme,'\n')
    
    w = 0.5
    oof2 = (1-w)*np.array(super_final_oof_predictions[-1]) + w*np.array(super_final_oof_predictions2[-1])
    rsme = 100*np.sqrt(np.mean((super_final_oof_true[-1] - oof2)**2.0))
    print('NN & NN+SVR Ensemble RSME =',rsme,'\n')

## CV score

In [ ]:
true = np.hstack(super_final_oof_true)

oof = np.hstack(super_final_oof_predictions)
rsme = 100*np.sqrt(np.mean((oof - true)**2.0))
print('Overall CV NN head RSME =',rsme)

oof2 = np.hstack(super_final_oof_predictions2)
rsme = 100*np.sqrt(np.mean((oof2 - true)**2.0))
print('Overall CV SVR head RSME =',rsme)

oof3 = (1-w)*oof + w*oof2
rsme = 100*np.sqrt(np.mean((oof3 - true)**2.0))
print('Overall CV Ensemble heads RSME with 50% NN and 50% SVR =',rsme)

## Best weight for SVR in ensemble

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

score = []
for ww in np.arange(0,1.05,0.05):
    oof3 = (1-ww)*oof + ww*oof2
    rsme = 100*np.sqrt(np.mean((oof3 - true)**2.0))
    #print(f'{ww:0.2} CV Ensemble RSME =',rsme)
    score.append(rsme)
best_w = np.argmin(score)*0.05

plt.figure(figsize=(20,5))
plt.plot(np.arange(21)/20.0,score,'-o')
plt.plot([best_w],np.min(score),'o',color='black',markersize=15)
plt.title(f'Best Overall CV RSME={np.min(score):.4} with SVR Ensemble Weight={best_w:.2}',size=16)
plt.ylabel('Overall Ensemble RSME',size=14)
plt.xlabel('SVR Weight',size=14)
plt.show()

## Submission

In [ ]:
super_final_predictions = np.mean(np.column_stack(super_final_predictions), axis=1)
super_final_predictions2 = np.mean(np.column_stack(super_final_predictions2), axis=1)

sample_df = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')
preds = (1-best_w)*super_final_predictions + best_w*super_final_predictions2
sample_df['Pawpularity'] = preds*100
sample_df.to_csv('submission.csv',index=False)
sample_df.head()